<a href="https://colab.research.google.com/github/adrian-alejandro/autoML/blob/main/AutoML_Practico_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

## Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2022

### AutoML

**Integrantes**:
- Anahí Sulca
- Adrián Zelaya

# Objetivo

Utilizando la librería Optuna, implementar optimización de hiperparámetros (HPO) sobre los modelos de aprendizaje automático desarrollados para resolver el problema de clasificación multi-clase de documentos legales por el fuero de pertenencia, usando de base el set de datos de la mentoría de [Búsqueda y Recomendación de Textos Legales](https://github.com/adrian-alejandro/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022).

# Descripción de los datos

Los datos utilizados en este trabajo fueron preprocesados bajo el marco de la mentoría [Búsqueda y Recomendación de Textos Legales](https://github.com/adrian-alejandro/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022).

El dataset consiste de documentos de textos correspondientes a fallos y sentencias judiciales del Poder Judicial de la Provincia de Córdoba, divididos en los siguentes fueros:

- Familia
- Laboral
- Menores
- Penal

Sobre los textos se realizaron las siguientes tareas de preprocesamiento:
- tokenización
- eliminación de stopwords
- eliminación de no-palabras (símbolos, puntuación, etc.)
- transformar a minúsculas
- lematización

Finalmente, el texto tokenizado y depurado es transformado a una representación vectorial TF-IDF (term-frequency - inverse-document-frequency) donde cada fila se corresponde con un documento y cada columna con el coeficiente TF-IDF de una data palabra para dicho documento.

En el suguiente [notebook](https://github.com/adrian-alejandro/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022/blob/main/Practico%203%20-%20Embeddings.ipynb) se encuentra detallada la implementación de los pasos mencionados en esta sección.


# Descripción de Optuna

Optuna es una estructura de software de optimización automática de hiperparámetros de código abierto, diseñado especialmente para el aprendizaje automático. Cuenta con una API de usuario imperativa define-by-run que garantiza que el código escrito con Optuna disfrute de una alta modularidad y que el usuario pueda construir dinámicamente los espacios de búsqueda para los hiperparámetros.

Entre las carácteristicas de Optuna podemos mencionar:
* Arquitectura agnóstica de la plataforma
* Espacios de búsqueda de Pythonic
* Algoritmos de optimización de última generación
* Paralelización
* Visualización (varias funciones de trazado).






# Configuración del entorno de trabajo

Instalamos/importamos las librerías necesarias

In [ ]:
!pip install optuna

In [ ]:
import os
from io import BytesIO
import requests
import pandas as pd
import numpy as np
import optuna

from sklearn import datasets
from sklearn import model_selection
from sklearn import svm, naive_bayes, linear_model
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import train_test_split

# Importación de los datos

Importamos los datos preprocesados, distribuidos en dos archivos:
- Matrix de representación TF-IDF (numpy array)
- Dataframe con información de los documentos del dataset

In [ ]:
vectorized_dataset_url = "https://raw.githubusercontent.com/adrian-alejandro/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022/main/embeddings/vectorized_dataset_X_y.npz"
processed_data_url = "https://raw.githubusercontent.com/adrian-alejandro/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022/main/embeddings/processed_dataset.csv"

In [ ]:
vector_file = os.path.split(vectorized_dataset_url)[1]
dataset = os.path.split(processed_data_url)[1]

In [ ]:
def read_npz_from_url(url):
  """Function that reads a npz file from a URL and overrides np.load method to
   force allow_pickle.
   Ref: https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa
  """
  # save np.load
  np_load_old = np.load

  # modify the default parameters of np.load
  np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

  r = requests.get(url, stream=True)

  # call load_data with allow_pickle implicitly set to true
  vectors = np.load(BytesIO(r.raw.read()))

  # restore np.load for future normal usage
  np.load = np_load_old

  return vectors

In [ ]:
vectors = read_npz_from_url(vectorized_dataset_url)

# Texto vectorizado
X = vectors['X']
                                            

# Etiquetas (fueros)
y = vectors['y']

Inspeccionamos la data:

In [ ]:
data = pd.read_csv(processed_data_url, sep='|', encoding='utf-8')
data.head()

,archivo,fuero,texto_clean
0,9 BAEZ-FLECHA BUS.pdf.txt,LABORAL,"['sala', 'laboral', 'tribunal', 'superior', 'p..."
1,90 FUNES-COYSPU.pdf.txt,LABORAL,"['sala', 'laboral', 'tribunal', 'superior', 'p..."
2,1 QUINTEROS-CONSOLIDAR.pdf.txt,LABORAL,"['sala', 'laboral', 'tribunal', 'superior', 'p..."
3,3 SANGUEDOLCE-MUNICIPALIDAD DE VILLA ALLENDE.p...,LABORAL,"['sala', 'laboral', 'tribunal', 'superior', 'p..."
4,188 LUCIANO-NICOLAS.pdf.txt,LABORAL,"['sala', 'laboral', 'tribunal', 'superior', 'p..."


Separamos el dataset en train/test. Utilizaremos el dataset de train para la HPO y luego el dataset de test para evaluar el mejor modelo.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

# Optimización de Hiperparámetros

Definimos la función objetivo que se utilizará para la HPO. En la misma se definieron:

*   los estimadores (clasificadores): Logistic Regression, SVM y Multinomial Naïve-Bayes
*   los hiperparámetros (search space) a optimizar para cada uno de los clasificadores
*   el método de scoring, que en nuestro caso es el balanced accuracy

Utilizamos el sampler default de Optuna, [TPE (Tree-structured Parzen Estimator)](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler), dado que permite trabajar con el tipo de parámetros que optimizaremos en nuestro caso (floats, categorical).

Definimos la dirección de la optimización como `maximize` dado que queremos maximizar nuestra métrica de evaluación (balanced accuracy).



In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = trial.suggest_categorical("classifier", ["LogReg", "SVM", "MNB"])
    
    # Step 2. Setup values for the hyperparameters:
    if classifier_name == 'LogReg':
      # Seach space
      logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
      logreg_l1_ratio = trial.suggest_float("logreg_l1_ratio", 0.0, 1.0)
      logreg_solver = trial.suggest_categorical("logreg_solver", ['newton-cg', 'lbfgs', 'sag', 'saga'])
      logreg_fit_intercept = trial.suggest_categorical("logreg_fit_intercept", [False, True])
      logreg_class_weight = trial.suggest_categorical("logreg_class_weight", ['balanced', None])
      logreg_penalty = trial.suggest_categorical("logreg_penalty", ['l1', 'l2', 'elasticnet', 'none'])

      # Estimator
      classifier_obj = linear_model.LogisticRegression(
          C=logreg_c,
          solver=logreg_solver,
          fit_intercept=logreg_fit_intercept,
          penalty=logreg_penalty,
          class_weight=logreg_class_weight,
          l1_ratio=logreg_l1_ratio)
    elif classifier_name == "SVM":
      # Search space
      svm_c = trial.suggest_float("svm_c", 1e-1, 1e1)
      svm_kernel = trial.suggest_categorical("svm_kernel", ['poly', 'linear', 'rbf', 'sigmoid'])
      svm_gamma = trial.suggest_categorical("svm_gamma", ['scale', 'auto'])
      svm_shrinking = trial.suggest_categorical("svm_shrinking", [True, False])
      svm_break_ties = trial.suggest_categorical("svm_break_ties", [True, False])
      svm_decision_function_shape = trial.suggest_categorical("svm_decision_function_shape", ['ovo', 'ovr'])
      # Estimator
      classifier_obj = svm.SVC(
            C=svm_c, 
            kernel=svm_kernel,
            gamma=svm_gamma,
            shrinking=svm_shrinking,
            break_ties=svm_break_ties,
            decision_function_shape=svm_decision_function_shape)
    else:
      # Search space
      mnb_alpha = trial.suggest_float("mnb_alpha", 1e-1, 1e1)
      mnb_fit_prior = trial.suggest_categorical("mnb_fit_prior", [True, False])
      # Estimator
      classifier_obj = naive_bayes.MultinomialNB(
          alpha=mnb_alpha,
          fit_prior=mnb_fit_prior)

    # Step 3: Scoring method:
    score = model_selection.cross_val_score(
        classifier_obj, 
        X_train,
        y_train,
        n_jobs=-1,
        cv=5,
        scoring=make_scorer(balanced_accuracy_score))
    balanced_accuracy = score.mean()
    return balanced_accuracy

Corremos nuestro estudio y seleccionamos el estudio con los mejores parámetros

In [ ]:
study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=50)

[I 2022-12-11 22:18:13,887] A new study created in memory with name: no-name-a7f85623-a8e3-4439-a9cb-22f852e97670
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/dist-pa

In [ ]:
best_trial = study.best_trial
best_params = {k.replace(best_trial.params['classifier'].lower() + '_', ''): v 
               for k,v in  best_trial.params.items() if k != 'classifier'}

print(f"El mejor score (balanced accuracy) es: {best_trial.value}")
print(f"El mejor clasificador es un {best_trial.params['classifier']} con los siguientes parámetros: \n{best_params}")


El mejor score (balanced accuracy) es: 0.9689338235294118
El mejor clasificador es un LogReg con los siguientes parámetros: 
{'c': 10.793634254833616, 'l1_ratio': 0.45378611277561787, 'solver': 'saga', 'fit_intercept': False, 'class_weight': 'balanced', 'penalty': 'elasticnet'}


Vemos la evolución de la optimización para los distintos trials. Los mejores valores se muestran en la curva 'Best Value', y se mantienen notablemente elevados. Se pueden observar también trials con un bajo desempeño (valores cercanos a cero), y en valores intermedios.

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study).show()

Probamos el mejor modelo con el set de evaluación para verificar los resultados. Notamos que el score se mantiene alto (99%), consistentemente con los scores obtenidos durante la optimización.

In [ ]:
# Best classifier: 'LogReg'
best_params

best_model = linear_model.LogisticRegression(
    C=best_params['c'],
    solver=best_params['solver'],
    fit_intercept=best_params['fit_intercept'],
    class_weight=best_params['class_weight'],
    penalty=best_params['penalty'],
    l1_ratio=best_params['l1_ratio']
)

best_model.fit(X_train, y_train)

y_predict = best_model.predict(X_test)

print(f"El score (balanced accuracy) del set de evaluación es:{balanced_accuracy_score(y_test, y_predict)}")

El score (balanced accuracy) del set de evaluación es:0.9939024390243902


# Conclusión

Se implementó una HPO de diversos estimadores para resolver un problema de clasificación multi-clase. 

Si bien los resultados obtenidos son similares a aquellos obtenidos aplicando un grid search o un random grid search, 98% (original) vs 97% (Optuna), los tiempos de ejecución son significativamente más reducidos utilizado Optuna: minutos vs horas (original).

Nos parece importante recalcar el tema del tiempo, dado que el dataset no es demasiado grande (< 250 fallos) y por lo tanto la implementación con Optuna nos permitiría escalar mejor si se llegara a ampliar el número de documentos del dataset.

Otra ventaja de utilizar Optuna es su simplicidad al momento de definir la función objetivo y el espacio de búsqueda.
